# Leaf Disease Classification using Pytorch with ResNet34 Pretrained Model

Binary Classification

### Importing the packages

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F

from sklearn import metrics, model_selection, preprocessing

## Data Loading

In [ ]:
dfx = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=42, stratify=dfx.label.values
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

image_path = "../input/cassava-leaf-disease-classification/train_images/"
train_image_paths = [os.path.join(image_path, x) for x in df_train.image_id.values]
valid_image_paths = [os.path.join(image_path, x) for x in df_valid.image_id.values]
train_targets = df_train.label.values
valid_targets = df_valid.label.values

### Shape of train set

In [ ]:
len(train_image_paths),len(train_targets)

### Shape of test set

In [ ]:
len(valid_image_paths),len(valid_targets)

## Building Dataset Class

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, targets, transform):
        super().__init__()
        self.files = data
        self.targets = targets
        self.classes = list(set(targets))
        self.transform = transform
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        name = self.files[i]
        fpath = os.path.join(name)
        img = self.transform(open_image(fpath))
        class_idx = self.targets[i]
        return img, class_idx

### Augmentation step

In [ ]:
import torchvision.transforms as T

img_size = 224
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
train_dataset = CustomDataset(train_image_paths,train_targets, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))

valid_dataset = CustomDataset(valid_image_paths,valid_targets, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))

### Plotting util functions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

def open_image(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
def denormalize(images, means, stds):
    if len(images.shape) == 3:
        images = images.unsqueeze(0)
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

def show_image(img_tensor, label):
    print('Label:', train_dataset.classes[label], '(' + str(label) + ')')
    img_tensor = denormalize(img_tensor, *imagenet_stats)[0].permute((1, 2, 0))
    plt.imshow(img_tensor)

### Train set images

In [ ]:
show_image(*train_dataset[0])

### Validation images

In [ ]:
# valid_image_paths[2139], valid_targets[2139]
show_image(*valid_dataset[2139])

## Building DataLoader

In [ ]:
from torch.utils.data import DataLoader
batch_size = 256

train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_dl = DataLoader(valid_dataset, batch_size*2, num_workers=4, pin_memory=True)

### Displaying Batch data

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(16, 16))
        ax.set_xticks([]); ax.set_yticks([])
        images = denormalize(images[:64], *imagenet_stats)
        ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl) #takes 2 min

### Defining Model and its Metrics

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}],{} train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, "last_lr: {:.5f},".format(result['lrs'][-1]) if 'lrs' in result else '', 
            result['train_loss'], result['val_loss'], result['val_acc']))

## Building Model

In [ ]:
from torchvision import models

class LeafModel(ImageClassificationBase):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=pretrained)
        # Replace last layer
        self.network.fc = nn.Linear(self.network.fc.in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)

#### Clearing gpu cached memory

In [ ]:
torch.cuda.empty_cache()

### Defining GPU Functions

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

## Model Fitting

In [ ]:
from tqdm.notebook import tqdm

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
device = get_default_device()
device

### Putting data into GPU

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

## Downloading Pretrained Resnet34 model

In [ ]:
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser

cache_dir = expanduser(join('~', '.torch'))
if not exists(cache_dir):
    makedirs(cache_dir)
    
cache_dir = expanduser(join('~', '.cache/torch/hub/checkpoints/'))
if not exists(cache_dir):
    makedirs(cache_dir)
    
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [ ]:
!cp -f ../input/resnet34/resnet34.pth /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth

### Model to GPU

In [ ]:
model = LeafModel(len(train_dataset.classes))
to_device(model, device);

### Evaluate base model

In [ ]:
history = [evaluate(model, valid_dl)]
# history

## Defining training parameters

In [ ]:
epochs = 6
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

### TRaining

In [ ]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

In [ ]:
# history

## Predict

In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return int(preds)

In [ ]:
img_tensor, label = valid_dataset[2]
show_image(*(img_tensor,label))
print('Label:', label, ', Predicted:', predict_image(img_tensor, model))

In [ ]:
img_tensor, label = valid_dataset[5]
show_image(*(img_tensor,label))
print('Label:', label, ', Predicted:', predict_image(img_tensor, model))

In [ ]:
img_tensor, label = valid_dataset[8]
show_image(*(img_tensor,label))
print('Label:', label, ', Predicted:', predict_image(img_tensor, model))

### Submission Ready

In [ ]:
test_dfx = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
image_path = "../input/cassava-leaf-disease-classification/test_images"


test_image_paths = [os.path.join(image_path, x) for x in test_dfx.image_id.values]
test_targets = test_dfx.label.values


test_dataset = CustomDataset(test_image_paths,test_targets, T.Compose([T.Resize(img_size), 
                                           T.Pad(8, padding_mode='reflect'),
                                           T.RandomCrop(img_size), 
                                           T.ToTensor(), 
                                           T.Normalize(*imagenet_stats)]))

In [ ]:
len(test_image_paths)

In [ ]:
final_preds = []
for i in test_dataset:
    img_tensor, label = i
    pred = predict_image(img_tensor, model)
    final_preds.append(pred)

In [ ]:
img_tensor, label = test_dataset[0]
show_image(*(img_tensor,label))
pred = predict_image(img_tensor, model)
print('Label:', label, ', Predicted:', pred)

In [ ]:
test_dfx.label = final_preds
test_dfx.to_csv('submission.csv', index=False)

# Please upvote if you like the work. :)